In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
train_path="data/dv/train"
test_path="data/dv/test"
val_path="data/dv/val"

In [ ]:
# Image preprocessing using VGG-19 model

# resize images to 224x224 and store in array

x_train=[]
for folder in os.listdir(train_path):
    if (folder != ".DS_Store"):
        sub_path=train_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr=cv2.resize(img_arr,(224,224))
                x_train.append(img_arr)

x_test=[]
for folder in os.listdir(test_path):
    if (folder != ".DS_Store"):
        sub_path=test_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr=cv2.resize(img_arr,(224,224))
                x_test.append(img_arr)
                
x_val=[]
for folder in os.listdir(val_path):
    if (folder != ".DS_Store"):
        sub_path=val_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr=cv2.resize(img_arr,(224,224))
                x_val.append(img_arr)
                

In [ ]:
# divide by 255 to normalize
train_x=np.array(x_train)/255.0
test_x=np.array(x_test)/255.0
val_x=np.array(x_val)/255.0


In [ ]:
# Compute labels of corresponding datasets
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

# generate batches of augmented data
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

In [ ]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [ ]:
training_set.class_indices
train_y.shape,test_y.shape,val_y.shape

In [ ]:
# Model Building
vgg = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# We will not train the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(3, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

In [ ]:
# Compile the model
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
# Fit the model

# To avoid overfitting, we will do an early stop if the loss substantially increases
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

train_y = train_y[1: len(train_y) - 1]

# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)